In [1]:
# imports
from aerofoil2 import main_pontoon_calc
from aerofoil import calc_naca_area


In [2]:
# define river speed and power requirements (the RPM is calculated from the river velocity)
foil_width = 0.3
river_vel = 1
turbine_width = 1
pontoon_length = 1.5


In [5]:
# apply effect of pontoons on river speed (calculate pontoon volume)
mean_pontoon_velocity = main_pontoon_calc(foil_width, turbine_width, pontoon_length,river_vel,plot=True)

/Users/philblecher/Desktop/Github/DP5-code/aerofoil2.py:290: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return integrate.quad(integrand, 0.0, panel.length)[0]


In [6]:
# calculate the area of the turbine
pontoon_area = calc_naca_area(foil_width)

In [7]:
print('Pontoon area: ', pontoon_area)
print('Mean pontoon velocity: ', mean_pontoon_velocity)

Pontoon area:  0.20533427225358855
Mean pontoon velocity:  1.1116021490311168


In [ ]:
# calculate RPM (using the velocity model for shaft power output)


In [ ]:
# with the RPM, power requirements, river velocity, and efficiency estimates for components
# calculate base dimensions for the turbine (width and radius of the turbine) (calculate rough volume of the turbine)


In [ ]:
# using the above calculate the required blade thickness (with additional input of material properties)

In [ ]:
# run the debris impact model to determine worst case debris size and impact force

In [ ]:
# using dimensions and inputs calculate the torque on the drive shaft